# THis is a test  
'''''' The script will also bring in ASSI data and find all "fields" within 3km of the selected ASSI.
        The script will then summarise a number of dummy field attributes, joined from a separate table to the 'agricultural fields (aka small areas)' data, within the 3km ASSI buffer area to provide a profile of the buffer area.
''''''


In [ ]:
##################################################### ALL ASSIs in NI###########################################################from cartopy.feature import ShapelyFeature
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy
import cartopy.io.img_tiles as cimgt
import geopandas as gpd
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import folium
#import ipywidgets
#from IPython.display import HTML, display
import numpy as np
from __future__ import division
import openpyxl

#- - - - - - - - - - - - - - -  - - - -Create an output map to show all ASSI's buffered - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#First ask user to determine the buffer distance to be used
#print("Specify a buffer distance in kilometers. \nInput a whole number:")
#Dist_km_str = input()
#Dist_km_int = int(Dist_km_str)
#Dist_m_int = Dist_km_int*1000

################################################# New functions ############################################################

# This "assign_Area_Length" function assigns area in km2 and length in meters to any geodataframe
def assign_Area_Length(gdf):
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        gdf.loc[ind, 'Area_km2'] = row['geometry'].area / 1000000 # assign the row's geometry length to a new column, Area
    
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        gdf.loc[ind, 'Length_m'] = row['geometry'].length  # assign the row's geometry length to a new column, Length

################################################# # adding and work with the ASSI data ############################################################
ASSI = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project//data_files/ASSI.shp')).to_crs(epsg=2157)
assign_Area_Length(ASSI) #apply user created function to assign lenght and area to gdf
ASSI = ASSI.drop(['MAP_SCALE', 'CONFIRMDAY', 'CONFIRM_HA', 'DECLAREDAY','DECLARE_HA',
                  'GIS_AREA', 'GIS_LENGTH', 'PARTIES', 'Shape_STAr', 'Shape_STLe', 'Hyperlink'], axis=1)    #Drop unneccessary fields

newASSIorder = ['OBJECTID','geometry', 'REFERENCE',  'NAME', 'COUNTY', 'SPECIESPT1', 'SPECIESPT2', #Create new order for columns
                  'HABITAT', 'EARTH_SCI', 'Area_km2', 'Length_m']                   
ASSI= ASSI.reindex(columns=newASSIorder)                                                           #Apply new order for columns
ALL_ASSI = ASSI

################################################# # adding and work with the Field data ############################################################
#Add first Agri dataset(inc. Field ID's and geometry)
AgFields = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project/data_files/AgFields.shp')) # add Agricultural fields data
AgFields.to_crs(epsg=2157, inplace=True)
assign_Area_Length(AgFields)             #apply user created function to assign length and area to gdf
newAgFieldsorder = ['geometry', 'FieldID',  'Hectares', 'Area_km2', 'Length_m', 'X_COORD', 'Y_COORD']  #Create new order for columns
                                       
AgFields= AgFields.reindex(columns=newAgFieldsorder)   

#Add second Agri dataset (inc.Field ID's and animal counts)
FieldInfo =  pd.read_excel(os.path.abspath('C:/Carol_PG_CERT_GIS/EGM722_Project/data_files/FieldInfo.xlsx')) #Add in excel table to join data

#join FieldInfo (pandas dataframe) onto the AgFields(geopandas dataframe)
AgFields = AgFields.merge(FieldInfo, on='FieldID', how='left')
AgFields = AgFields.drop(['X_COORD', 'Y_COORD'], axis=1) # drop unrequired columns

################################################# # adding and work with the LandUse data################################################


In [ ]:
noLS = AgFields.loc[(AgFields["Cattle_Count"] == 0) & (AgFields['Pig_Count'] == 0) & (AgFields['PoultryHouses'] == 0)]
FieldCount = noLS.shape[0]
noLS_FieldArea = noLS.noLS['Area_km2'].sum()
#print("The total number of fields with no cattle, sheep or poultry houses is : " + str(FieldCount))
#print("The total area of agricultural fields with no cattle, sheep or poultry houses is {:.2f} km2".format(noLS_FieldArea)

In [ ]:
#ALL_ASSI_buffer['geometry'].ALL_ASSI_buffer['NAME' == 'Lough Neagh']   # will show you  the shape of the feature on the first row
#ALL_ASSI_buffer['geometry'].iloc[(ALL_ASSI_buffer['NAME'] == 'Lough Neagh')]
AgFields_Area =AgFields['Area_km2'].sum()
ASSI_Area =ASSI['Area_km2'].sum()

print("Total area of agricultural fields {:.2f} km2".format(AgFields_Area))
print("Total area of agricultural fields {:.2f} km2".format(AgFields_Area))
print("Total area of agricultural fields {:.2f} km2".format(AgFields_Area))
print("Total area of agricultural fields {:.2f} km2".format(AgFields_Area))

In [ ]:
noLS.head()


In [ ]:
Poultry_Summary= AgFields.groupby(['PoultryHouses'])['Area_km2'].sum() # assign summary data to variable to be printed
print(Poultry_Summary)

In [ ]:
joinedData = join.groupby(['CountyName'])['Population'].sum() # assign summary data to variable to be printed

In [ ]:
#- - - - - - - - -  - - - - - - - - - interrogate data


#a=farmCensusDEA.loc[(farmCensusDEA['TotalCrops']   < 100) & (farmCensusDEA['OBJECTID']     < 10), "TotalCrops"].sum()
lakes.head(-1)
lakes.loc[lakes["town"] == "THE DIAMOND"] #select based on 1: gdf.loc  [gdf['ColumnHeading'< 0] 
lakes.loc[(lakes["town"] == "THE DIAMOND") & (lakes['name'] == "Lough Neagh")] #select based on 2: 
gdf.loc[(gdf["colum"] == ("value") or gdf["colums"] == ("value"))]
lakes.loc[(lakes["town"] == "THE DIAMOND") & (lakes["county"] == "Tyrone") & (lakes['name'] == "Lough Neagh")]




print(AgFields['TotalAreaF'].mean())
print(farmCensusDEA['TotalCrops'].sum())

In [ ]:
FieldsInBuffer = gpd.sjoin(AgFields,ALL_ASSI_buffer,how='inner',predicate='intersects')

In [ ]:
#Output stats
ALL_ASSI_buffer.head() # total area falling within the BUFFER
print("Total area falling within the buffer {:.2f} km2".format(total_Area))

In [ ]:
ALL_ASSI_buffer.describe()

In [ ]:
varname2 = ALL_ASSI_buffer[ALL_ASSI_buffer["Area_km2"] != " "]
varname2.describe


In [ ]:
varname2 = ALL_ASSI_buffer[ALL_ASSI_buffer["Area_km2"] != " "]
varname2.describe()


In [ ]:
#- - - - - - - - -  - - - - - - - - - - -  Present results from analysis- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#to get the top 10 values of a field:
top_10 = FieldsInBuf.sort_values("GIS_AREA", ascending=False).head(10)
top_10

#TODO: present results
#print(type(top_10))
#top_10feature = ShapelyFeature(top_10['geometry'], myCRS, edgecolor='yellow', facecolor='none', linewidth=2)
#ax.add_feature(top_10feature)
#myFig

In [ ]:
#-----------------------------How to get poly area/length bound geometry:-----------------------------------
#print('perimeter: ', poly1.length) # print the perimeter
#print('area: ', poly1.area) # print the area
#print('centroid: ', poly1.centroid) # get the centerpoint of the rectangle

In [ ]:
#-------------------How to Return whole row for a fetaure with max value:------
#df[df['Value']==df['Value'].max()]

#-------------------or#-------------------
gdf["columnName"].value_counts
gdf[gdf["columnName"]=='avalue']["columnName"].sum
max_of something = gdf["columnName"].max
gdf['column'] == max.area # to show the details of the row whtat correcpons to the max

In [ ]:
gdf['geometry'].iloc[0]   # will show you  the shape of the feature on the first row
str(gdf['geometry'].iloc[0]) # will show you  the string of coords of the feature on the first row

In [ ]:
#to describe values within a colum
varname = gdf[gdf["columnName"] != " "] #select records that are not empty
varname.describe() #prvide decriptive stats for these records- eg if you want stats for area or animal count


In [ ]:
#-----------------------------How to select features:--------------------------------------------------------------------------
    
#gdf.head()
#gdf.loc[0:2] #shows the first 3 rows
#gdf.loc[farmCensusDEA['ColumnName'] < 100]#shows total crops <100 gdf.loc[gdf['ColumnHeading'] 
#to select records based upon 2 variable: 
#gdf.loc[(gdf['columheading1']> 435) & (dgf['columheading2'] < 1)]
#gdf.loc[(gdf['ColumnName']   < 100) & (gdf['ColumnName']     < 10)]
          
#to summarise a subset of records based upon 2 variable: 
#gdf.loc[(gdf['columheading1']> 435) & (gdf['columheading1'] < 1), 'ColumnToSummarise'].sum()
#gdf.loc[(gdf['ColumnName']   < 100) & (gdf'ColumnName']     < 10), 'ColumnName'].sum()
#a=gdf.loc[(gdf['ColumnName']   < 100) & (gdf['ColumnName']     < 10), 'ColumnName'].sum()

#print("Total x in areas that meet conditions ={}".format(a.astype(int)))


In [ ]:
#---------How to summarise features/selected feature attributes:----------------

#new var = df['columheading'].sum example below- this sums all records
#sum_roads = roads_itm['Length'].sum()

#below- this sums a selection of records
#new var = df[df['columheading'] == 'Value']['Fieldto Summarise'].sum example 
#sum_motorway = roads_itm[roads_itm['Road_class'] == 'MOTORWAY']['Length'].sum()
#print('{:.2f} total m of roads'.format(sum_roads))
#print('{:.2f} total m of motorway'.format(sum_motorway))

In [ ]:
#-----How to summarise features/selected feature attributes using GROUPBY:-----
#GroupBy one field:
#gdf.groupby(['FieldtoGroupby'])['FieldToSumariseBy'].sum() - example below
#roads_itm.groupby(['Road_class'])['Length'].sum() / 1000 # convert to km

#GroupBy two fields:
#gdf.groupby(['GroupByField1', 'GroupByField2'])['Length'].sum() / 1000- example below:
#join.groupby(['CountyName', 'Road_class'])['Length'].sum() / 1000


#GroupBY one field using selected records only:

#gdf[gdf['SelectionField'] == 'Value'].groupby(['GroupByField'])['SummaryField'].sum() 
# example below:
#roads_itm[roads_itm['Road_class'] == 'MOTORWAY'].groupby(['Road_class'])['Length'].sum() / 1000 # convert to km

#GroupBY two fields using selected records only:
#gdf[gdf['SelectionField'] == 'Value'].groupby(['Field1', 'Field2'])['FieldToSummarise'].sum() 

In [ ]:
#to write data outputs to an excel worksheet
wbk = xlsxwriter.Workbook('testing.xlsx')
wks = wbk.add_worksheet()
wks.write('A1', 'Hello world')
print (wks.name)
wks2 = wbk.add_worksheet()
print (wks2.name)

i = -1

for x in range(1, 1000, 11):
    i+=1
    cella = xl_rowcol_to_cell(i, 0) #0,0 is A1!
    cellb = xl_rowcol_to_cell(i, 1)
    cellc = xl_rowcol_to_cell(i, 2)
    print (cella)
    wks2.write(cella,x)
    wks2.write(cellb,x*3)
    wks2.write(cellc,x*4.5)
wbk.close()

In [ ]:
FieldsInBuffer = gpd.sjoin(AgFields,ALL_ASSI_buffer,how='inner',predicate='intersects')

In [ ]:
FieldInfo = pd.read_excel(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project//data_files/FieldInfo.xlsx'))
#join a csv with a shapefile using a joinfield

new_gdf =gpd.GeoDataFrame(AgFields.merge(FieldInfo, on="Field"), index=True)
new_gdf.head()                                         

In [ ]:
#- - - - - - - - -  - - - - - - - - - - -  Present results from analysis- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#to get the top 10 values of a field:
top_10 = SelectedASSI.sort_values("GIS_AREA", ascending=False).head(10)
top_10

#TODO: present results
#print(type(top_10))
#top_10feature = ShapelyFeature(top_10['geometry'], myCRS, edgecolor='yellow', facecolor='none', linewidth=2)
#ax.add_feature(top_10feature)
#myFig

In [ ]:
intersectLayer = gpd.overlay(AgFields, SelectedASSI_buffer, how ='intersection')
intersectLayer2= intersectLayer['geometry'].unary_union 
#ASSI_buffer.head()
#ASSI_buffer.head()
print(type(intersectLayer))
intersectLayer_feature = ShapelyFeature(intersectLayer2['geometry'], myCRS, edgecolor='green', facecolor='none', linewidth=2)
ax.add_feature(intersectLayer_feature)

In [ ]:
for item in gdf.columnns[:-1]:  #we're not asking for the last column, as geometry is not iterable
    uniquelist = gdf[item].unique()
    if unique list.flatten().shape[0] < 10:       #we only what to know where there are less than 10 unique values in a colum
        print('Count name: {}'.format(item))
        print('Unique values: {}'.format(gdf[item].unique()))
        print('----------')
    
    

In [ ]:
#to describe values within a colum
varname = gdf[gdf["columnName"] != " "] #select records that are not empty
varname.describe() #prvide decriptive stats for these records- eg if you want stats for area or animal count


gdf.total_bounds # to get the bounding coords


In [ ]:
gdf["columnName"].value_counts
gdf[gdf["columnName"]=='avalue']["columnName"].sum
max_of something = gdf["columnName"].max
gdf['column'] == max.area # to show the details of the row whtat correcpons to the max



In [ ]:
ALL_ASSI_buffer[ALL_ASSI_buffer['NAME' == 'Lough Neagh']["Area_km"]].sum




In [ ]:
gdf['geometry'].iloc[0]   # will show you  the shape of the feature on the first row
str(gdf['geometry'].iloc[0]) # will show you  the string of coords of the feature on the first row

In [ ]:
print(farmCensusDEA.shape) # 80 rows and 28 fields
print(farmCensusDEA.columns)
print(farmCensusDEA['TotalAreaF'].mean())
print(farmCensusDEA['TotalCrops'].sum())

In [ ]:

~~~~~Assign summary data to w/s in excel


In [ ]:
##################################################### ALL ASSIs in NI###########################################################from cartopy.feature import ShapelyFeature
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy
import cartopy.io.img_tiles as cimgt
import geopandas as gpd
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import folium
#import ipywidgets
#from IPython.display import HTML, display
import numpy as np
from __future__ import division
import openpyxl

#- - - - - - - - - - - - - - -  - - - -Create an output map to show all ASSI's buffered - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#First ask user to determine the buffer distance to be used
print("Specify a buffer distance in kilometers. \nInput a whole number:")
Dist_km_str = input()
Dist_km_int = int(Dist_km_str)
Dist_m_int = Dist_km_int*1000

################################################# New functions ############################################################

# This "assign_Area_Length" function assigns area in km2 and length in meters to any geodataframe
def assign_Area_Length(gdf):
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        gdf.loc[ind, 'Area_km2'] = row['geometry'].area / 1000000 # assign the row's geometry length to a new column, Area
    
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        gdf.loc[ind, 'Length_m'] = row['geometry'].length  # assign the row's geometry length to a new column, Length

################################################# # adding and work with the ASSI data ############################################################
ASSI = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project//data_files/ASSI.shp')).to_crs(epsg=2157)
assign_Area_Length(ASSI) #apply user created function to assign lenght and area to gdf
ASSI = ASSI.drop(['MAP_SCALE', 'CONFIRMDAY', 'CONFIRM_HA', 'DECLAREDAY','DECLARE_HA',
                  'GIS_AREA', 'GIS_LENGTH', 'PARTIES', 'Shape_STAr', 'Shape_STLe', 'Hyperlink'], axis=1)    #Drop unneccessary fields

newASSIorder = ['OBJECTID','geometry', 'REFERENCE',  'NAME', 'COUNTY', 'SPECIESPT1', 'SPECIESPT2', #Create new order for columns
                  'HABITAT', 'EARTH_SCI', 'Area_km2', 'Length_m']                   
ASSI= ASSI.reindex(columns=newASSIorder)                                                           #Apply new order for columns
ALL_ASSI = ASSI

################################################# # adding and work with the Field data ############################################################
#Add first Agri dataset(inc. Field ID's and geometry)
AgFields = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project/data_files/AgFields.shp')) # add Agricultural fields data
AgFields.to_crs(epsg=2157, inplace=True)
assign_Area_Length(AgFields)             #apply user created function to assign length and area to gdf
newAgFieldsorder = ['geometry', 'FieldID',  'Hectares', 'Area_km2', 'Length_m', 'X_COORD', 'Y_COORD']  #Create new order for columns
                                       
AgFields= AgFields.reindex(columns=newAgFieldsorder)   

#Add second Agri dataset (inc.Field ID's and animal counts)
FieldInfo =  pd.read_excel(os.path.abspath('C:/Carol_PG_CERT_GIS/EGM722_Project/data_files/FieldInfo.xlsx')) #Add in excel table to join data

#join FieldInfo (pandas dataframe) onto the AgFields(geopandas dataframe)
AgFields = AgFields.merge(FieldInfo, on='FieldID', how='left')
AgFields = AgFields.drop(['X_COORD', 'Y_COORD'], axis=1) # drop unrequired columns



#####LandUse data############################




#################Initial spatial analysis to create a user defined buffer of all ASSIs and export as a shapefile###########################

ALL_ASSI_buffer = ALL_ASSI.copy()
ALL_ASSI_buffer.geometry = ALL_ASSI.geometry.buffer(Dist_m_int) #create a buffer of ALL ASSIs based upon the User selected Buffer Distance
ALL_ASSI_buffer_dis = ALL_ASSI_buffer.dissolve()


ALL_ASSI_buffer_dis.to_file('All_ASSI_'+ str(Dist_km_int)+'kmBuf.shp') # exports the features to a new shapefile for future use (if required)


################# Analyse fields in relation to buffer #################

#--------------------------------------SAve data:----------------------------------------------------------------------------------

#GDF.to_excel("ALL_ASSI"+ str(Dist_km_int)+ "km.txt")
#FieldsIn3kmBuf.to_csv('test', index =True)
 
 ######################################################## MAke map##############################################################
   
# make the plotting interactive
%matplotlib notebook           

# create the applicable CRS -in this case- Universal Transverse Mercator reference system- to transform the data
myCRS = ccrs.epsg(2157)

# create a figure of size 10x10 (representing the page size in inches)
myFig = plt.figure(figsize=(10, 10)) 

# create an axes object in the figure (within which the data shall be plotted), using the predefined crs
ax = plt.axes(projection=myCRS)  

#then get this dataset bounds and set them against the axes extent
xmin, ymin, xmax, ymax = AgFields.total_bounds 
ax.set_extent([xmin-6500, xmax+6500, ymin-6500, ymax+6500], crs=myCRS)

###- - - - - - - - - - - - - - -  Add background features to map (for aesthetic/contest setting purposes only) - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - -
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND)

   
# first, we add Agricultural fields data to map  :
AgFields_feature = ShapelyFeature(AgFields['geometry'], myCRS, edgecolor='w', facecolor='lightgrey', linewidth=0.3)
ax.add_feature(AgFields_feature)

# to add the ASSI data to map:
ALL_ASSI_feature = ShapelyFeature(ALL_ASSI['geometry'], myCRS, edgecolor='lightgreen', facecolor='lightgreen', linewidth=0.1)
ax.add_feature(ALL_ASSI_feature)


# add ASSI 3km Buffer polygons
ALL_ASSI_buffer_feature = ShapelyFeature(ALL_ASSI_buffer_dis['geometry'], myCRS, edgecolor='red', facecolor='none',
                                         hatch='xxx', 
                                         alpha=0.2, linewidth=1)
ax.add_feature(ALL_ASSI_buffer_feature)
ALL_ASSI_buffer_feature = ShapelyFeature(ALL_ASSI_buffer_dis['geometry'], myCRS, edgecolor='red', facecolor='none',linewidth=1)
ax.add_feature(ALL_ASSI_buffer_feature)


#add towns  to give additional context
towns = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/egm722_Practicals/egm722/week2/data_files/Towns.shp'))
towns.to_crs(epsg=2157, inplace=True)

# to add the town point data to map and create town handle for legend:
town_handle = ax.plot(towns.geometry.x, towns.geometry.y, 's', color='0.5', ms=6, transform=myCRS)
# add the text labels for the towns
for ind, row in towns.iterrows():  # towns.iterrows() returns the index and row
    x, y = row.geometry.x, row.geometry.y  # get the x,y location for each town
    ax.text(x, y, row['TOWN_NAME'].title(), fontsize=8, transform=myCRS)  # use plt.text to place a label at x,y

###- - - - - - - - - - - - - - - Create a map legend- - - - - - - - - - - - - - -
# generate a list of handles for the ASSI datasets
ALL_ASSI_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='lightgreen', edgecolor='lightgreen')]
ALL_ASSI_buffer_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='none', edgecolor='r')]
AgField_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='lightgrey', edgecolor='w')]

handles =  town_handle +  AgField_handles + ALL_ASSI_handles + ALL_ASSI_buffer_handles 
labels = ['Town', 'Field','ASSI', (str(Dist_km_int))+'km Buffer']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                fontsize=10, loc='upper left', frameon=True, framealpha=1)


#-----------------------------add gridlines:----------------------------------------------------------------------------------
gridlines = ax.gridlines(draw_labels=True,  # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],  # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5])  # add latitude lines at 0.5 deg intervals
gridlines.left_labels = True  # turn off the left-side labels
gridlines.bottom_labels = True  # turn off the bottom labels
gridlines.right_labels = False  # turn off the left-side labels
gridlines.top_labels = False  # turn off the bottom labels


#-----------------------------add a north arrow:----------------------------------------------------------------------------------
x, y, arrow_length = 0.93, 0.1, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor="black", width=3.5, headwidth=15),
            ha='center', va='center', fontsize=15,
            xycoords=ax.transAxes)


#-----------------------------add a title:--------------------------------------------------------------------------------    --
ax.set_title("Areas within " + str(Dist_km_int)+ "km Buffer of an ASSI");


#-----------------------------add a scale bar:--------------------------------------------------------------------------------   
def scale_bar(ax, location=(0.1, 0.05)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    #sbx = x0 + (x1 - x0) * location[0]
    #sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=7, transform=ax.projection)
    ax.plot([sbx, sbx - 5000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-5000, sbx - 10000], [sby, sby], color='w', linewidth=6, transform=ax.projection)
    ax.plot([sbx-9000, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-4500, '10 km', transform=ax.projection, fontsize=6)
    ax.text(sbx-4250, sby-4500, '5', transform=ax.projection, fontsize=6)#should be 6250 but doesn't place well
    ax.text(sbx-9750, sby-4500, '1', transform=ax.projection, fontsize=6)# should be 11250 but doesn't place well
    ax.text(sbx-12500, sby-4500, '0', transform=ax.projection, fontsize=6)# add the scale bar to the axis
scale_bar(ax)


#--------------------------------------Save figure:----------------------------------------------------------------------------------
# save the figure
mapNAme = 'All_ASSI'+'_'+ str(Dist_km_int)+'kmBuf_map.png'
myFig.savefig(mapNAme, dpi=300, bbox_inches='tight')

In [ ]:

~~~~~Assign summary data to w/s in excel


In [ ]:
##################################################### Selected ASSI###########################################################
####################################### Import required modules/packages/dependencies##########################################
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy
import cartopy.io.img_tiles as cimgt
import geopandas as gpd
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import folium
import ipywidgets
#from IPython.display import HTML, display
import numpy as np
from __future__ import division
import openpyxl
   
################################################# Essential User Input ######################################################
#first ask for user input to select an ASSI to investigate
print("Name of ASSI to investigate:")
Selected_ASSI = input()

#then ask user to determine the buffer distance to be used
print("Specify a buffer distance in kilometers. \nInput a whole number:")
Dist_km_str = input()
Dist_km_int = int(Dist_km_str)
Dist_m_int = Dist_km_int*1000

################################################# New functions ############################################################

# This "assign_Area_Length" function assigns area in km2 and length in meters to any geodataframe
def assign_Area_Length(gdf):
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        ASSI.loc[ind, 'Area_km2'] = row['geometry'].area / 1000000 # assign the row's geometry length to a new column, Area
    
    for ind, row in gdf.iterrows(): # iterate over each row in the GeoDataFrame
        ASSI.loc[ind, 'Length_m'] = row['geometry'].length  # assign the row's geometry length to a new column, Length

    
################################################# Initial Data SetUp ############################################################

%matplotlib notebook

# create the applicable CRS -in this case- Universal Transverse Mercator reference system- to transform the data
myCRS = ccrs.epsg(2157)

#- - - - - - - - - - - - - - - - - - - - Bring in the key spatial datasets- - - - - - - - - - - - - - - - - - - - - - - - 
# adding the ASSI data
ASSI = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project//data_files/ASSI.shp')).to_crs(epsg=2157)


# add Agricultural fields data
AgFields = gpd.read_file(os.path.abspath('c:/Carol_PG_CERT_GIS/EGM722_Project/data_files/AgFields.shp'))
AgFields.to_crs(epsg=2157, inplace=True)


#- - - - - - - - - - - - - - -   - - - - -  Bring in the key other datasets-- - - - - - - - - - - - - - - - - - - - - - - - -
#where we want to bring in field data
#~~~~~join other data table

#where we want to bring in landuse data
#where we want to join this data to the ASSI

#- - - - - - - - - - - - - - -   - - - - -  Create Essential layers for analyses- - - - - - - - - - - - - - - -- - - ---------
#create an GDF based upon the User selected ASSI
SelectedASSI = ASSI.loc[ASSI['NAME'] == Selected_ASSI]

#create an ASSI Buffer GDF based upon the User selected Buffer Distance
SelectedASSI_buffer = SelectedASSI.copy()
SelectedASSI_buffer.geometry = SelectedASSI.geometry.buffer(Dist_m_int)



#TOD remove spaces from ASSI Names for future saves
####shpName = Selected_ASSI[REFERENCE]+'_buf3km.shp' 
#SelectedASSI_buffer.to_file(shpName)                  # exports the features to a new shapefile for future use (if required)

################################################## Spatial Analysis #########################################################

#- - - - - - - - - - - - - -  -  - - create an gdf to identify fields within the buffer - - - - - - - - - -  - - - - - - - - - - - 
FieldsInBuf = gpd.sjoin(AgFields,SelectedASSI_buffer,how='inner',predicate='intersects',) 

#Create relevant filename (based on ASSI Reference) and save to shapefile
ASSICode = str(FieldsInBuf.REFERENCE.unique())
ASSICode = ASSICode.replace("[", "")
ASSICode = ASSICode.replace("]", "")
ASSICode = ASSICode.replace("'","")
FileTitle = (ASSICode+"_"+ str(Dist_km_int)+ "km"+".shp")  
FieldsInBuf.to_file(FileTitle)


#- - - - - - - - - - - -  Output summary analysis/ profile statistics for fields within the buffer area- - - - - - - - - - - -

#assign_Area_Length(ASSI) #apply user created function to assign lenght and area to gdf
#ASSI = ASSI.drop(['MAP_SCALE', 'CONFIRMDAY', 'CONFIRM_HA', 'DECLAREDAY','DECLARE_HA',
                  #'GIS_AREA', 'GIS_LENGTH', 'PARTIES', 'Shape_STAr', 'Shape_STLe'], axis=1)    #Drop unneccessary fields
#neworder = ['geometry','OBJECTID', 'REFERENCE',  'NAME', 'COUNTY', 'SPECIESPT1', 'SPECIESPT2', #Create new order for columns
                  #'HABITAT', 'EARTH_SCI', 'Areakm2', 'Length_m','Hyperlink']                   
#ASSI= ASSI.reindex(columns=neworder)                                                           #Apply new order for columns



######################## Create output map o show ASSI, Buffer extent and Fields within Buffer extent #########################

%matplotlib notebook
# make the plotting interactive
#plt.ion()

# create the applicable CRS -in this case- Universal Transverse Mercator reference system- to transform the data
myCRS = ccrs.epsg(2157)

# create a figure of size 10x10 (representing the page size in inches)
myFig = plt.figure(figsize=(10, 10)) 

# create an axes object in the figure (within which the data shall be plotted), using the predefined crs
ax = plt.axes(projection=myCRS)  

# first, we set the map extent
xmin, ymin, xmax, ymax = SelectedASSI.total_bounds #then get this dataset bounds and set them against the axes extent '
mapExtent = ax.set_extent([xmin-Dist_m_int-5000, xmax+Dist_m_int+5000, ymin-Dist_m_int-5000, ymax+Dist_m_int+5000], crs=myCRS)


###- - - - - - - - - - - - - - -  Add background features to map (for aesthetic/contest setting purposes only) - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - -

#ax.add_feature(cartopy.feature.OCEAN)
####request = cimgt.OSM()
####ax.add_image(request, 8)

###- - - - - - - - - - - - - - -  Add required data to the map- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -- - - -

#Create Shapely features to add to the map:
SelectedASSI_feature = ShapelyFeature(SelectedASSI['geometry'], myCRS, edgecolor='none', facecolor='lightgreen', linewidth=1)
SelectedASSI_buffer_feature = ShapelyFeature(SelectedASSI_buffer['geometry'], myCRS, edgecolor='red', facecolor='none', linewidth=1)
FieldsInBuf_feature = ShapelyFeature(FieldsInBuf['geometry'], myCRS, edgecolor='yellow', facecolor='none', linewidth=1)
AgFields_feature = ShapelyFeature(AgFields['geometry'], myCRS, edgecolor='w', facecolor='lightgrey', linewidth=0.3)

#Add Shapely features to the map:
ax.add_feature(AgFields_feature)               # add the Field data to map
ax.add_feature(SelectedASSI_feature)           # add the ASSI data to map
ax.add_feature(FieldsInBuf_feature)            # add the Selected Field data to map
ax.add_feature(SelectedASSI_buffer_feature)    # add selected ASSI 3km Buffer to map:
   
###- - - - - - - - - - - - - - - ------ add lables to the map- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - --
#add ASSI label to map- this first requires x,yco-ords to be assigned to the feature(s)
ASSI["x"] = ASSI.centroid.map(lambda p: p.x)
ASSI["y"] = ASSI.centroid.map(lambda p: p.y)
for ind, row in ASSI.loc[ASSI['NAME'] == Selected_ASSI].iterrows():  # ASSI.iterrows() returns the index and row
    x, y = row.x, row.y  # get the x,y location for each town
    ax.text(x, y, row['NAME'].title(), fontsize=8, color ='red', transform=myCRS)  # use plt.text to place a label at x,y
    

###- - - - - - - - - - - - - - - - - - --add legend  to the map- - - - - - - - - - - - - - -- - - - - - - -- - - - - - - -

# create handles for the legend:
ASSI_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='lightgreen', edgecolor='lightgreen')]
ASSI_buffer_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='none', edgecolor='r')]
AgField_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='lightgrey', edgecolor='w')]

handles =  AgField_handles + ASSI_handles + ASSI_buffer_handles  #generate a list of handles for the legend
labels = ['Field','ASSI', (str(Dist_km_int)+'km Buffer')]        #generate a list of lables for the legend

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                fontsize=10, loc='upper left', frameon=True, framealpha=1) #assign legend paramenters to variable 'leg'


#-----------------------------------------add gridlines:-----------------------------------------------------------------------

gridlines = ax.gridlines(draw_labels=True,  # draw  labels for the grid lines
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],  # add longitude lines at 0.5 deg intervals
                         ylocs=[54, 54.5, 55, 55.5])  # add latitude lines at 0.5 deg intervals
gridlines.right_labels = False  # turn off the left-side labels
gridlines.top_labels = False  # turn off the bottom labels
gridlines.left_labels = True  # turn on the left-side labels
gridlines.bottom_labels = True  # turn on the bottom labels
    
#-----------------------------------------add a scale bar:-----------------------------------------------------------------------
        
#TODO - FIx Scale bar This "scale_bar" function creates a scale bar 10km wide, broken down by 1km and 5km
def scale_bar(ax, location=(0.1, 0.05)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    #sbx = x0 + (x1 - x0) * location[0]
    #sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=7, transform=ax.projection)
    ax.plot([sbx, sbx - 5000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-5000, sbx - 10000], [sby, sby], color='w', linewidth=6, transform=ax.projection)
    ax.plot([sbx-9000, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-4500, '10 km', transform=ax.projection, fontsize=6)
    ax.text(sbx-4250, sby-4500, '5', transform=ax.projection, fontsize=6)#should be 6250 but doesn't place well
    ax.text(sbx-9750, sby-4500, '1', transform=ax.projection, fontsize=6)# should be 11250 but doesn't place well
    ax.text(sbx-12500, sby-4500, '0', transform=ax.projection, fontsize=6)# add the scale bar to the axis
scale_bar(ax)

#-----------------------------add a north arrow:----------------------------------------------------------------------------------

x, y, arrow_length = 0.93, 0.1, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor="black", width=3.5, headwidth=15),
            ha='center', va='center', fontsize=15,
            xycoords=ax.transAxes)


#-----------------------------add a title:--------------------------------------------------------------------------------    --

ax.set_title("Fields within " + str(Dist_km_int)+ "km Buffer of "+ Selected_ASSI +" ASSI");

#------------------------------save the figure: ---------------------------------------------------------------------------------

mapNAme = Selected_ASSI+'_'+ str(Dist_km_int)+'km''_map.png'
myFig.savefig(mapNAme, dpi=300, bbox_inches='tight')
#FieldsInBuf.to_csv(Selected_ASSI+ str(Dist_km_int)+ "km.txt", index =True)

#FieldsInBufdf = gdf.to_csv(Selected_ASSI+ str(Dist_km_int)+ "km.txt", index =True)

# then for the conversion, from geopandas to pandas I drop the last column (geometry) and specify the column names for the new df
#FieldsInBufdf = pd.DataFrame(FieldsInBuf.iloc[:,:-1].values, columns = list(FieldsInBuf.columns.values)[:-1] )
#FieldsInBufdf.to_csv(Selected_ASSI+ str(Dist_km_int)+ "km.txt", index =True)
FieldsInBuf.to_excel(Selected_ASSI+ str(Dist_km_int)+ "kmFields3.xlsx")

In [ ]:
#-----------------------------How to create LU tables:------------------------
# one way os to produce a dictionary object of key/value pairs:
# dict = {1:a,
#         2:b,
#         3:c}
#
#you can define a function to take an array and return a dictionary object- 
#containing the count for wach value in the array:
#def count_unique(array, names, nodata=0):
#    '''
#    Count the unique elements of an array.

#    :param array: Input array
#    :param names: a dict of key/value pairs that map raster values to a name
#    :param nodata: nodata value to ignore in the counting
    
#    :returns count_dict: a dictionary of unique values and counts
#    '''
#    count_dict = dict() # create the output dict
#    for val in np.unique(array): # iterate over the unique values for the raster
#        if val == nodata: # if the value is equal to our nodata value, move on to the next one
#            continue
#        count_dict[names[val]] = np.count_nonzero(array == val)
#    return count_dict # return the now-populated output dict
#
#newDict = count_unique(gdf, fieldname)
#print(newDict) # show the results
#######
#
#
#or try:
#county_dict = dict()
#for ind, row in counties.iterrows():
#    county_dict[row['CountyName'].title()] = county_stats[ind] # we're using str.title() because we're not shouting.
#
#print(county_dict['Down']) # this should be the same as the output for the previous cell
#
#
#or see Prac5 ipynb (section3) to use the built in zip function & https://docs.python.org/3.8/library/functions.html#zip

#Working with dictionary - see pract5 ipynb